In [1]:
download("https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/tags/train.txt", "train.txt")

In [2]:
download("https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/tags/dev.txt", "dev.txt")

In [3]:
using Pkg;Pkg.add("DataStructures")
Pkg.add("Dates")
Pkg.add("StatsBase")

In [1]:
using Knet: KnetArray,Knet, AutoGrad, param, param0, mat, RNN, relu, Data, adam, progress, nll, zeroone, logp, softmax
using DataStructures: DefaultDict
using IterTools: ncycle, takenth
using Random:randsubseq
using Random
using StatsBase
using Dates
using Random: shuffle!
using Base.Iterators: flatten
using Dates

┌ Warning: Hardware is unsupported by NNPACK so falling back to default NNlib
└ @ NNlib /kuacc/users/aalabrash18/.julia/packages/NNlib/Nksco/src/nnpack/NNPACK.jl:49


In [2]:
NXENT = 40
NXER = 20
delta = 2
EMBED_SIZE = 64
HIDDEN_SIZE = 128

128

In [3]:
w2i=Dict()
t2i=Dict()
function read_file(fname)
    input_file= readlines(fname)
    results=[]
    for row in input_file
        #@show split(row)
        words, tags = Array{Int32,1}(), Array{Int32,1}()
        for wt in split(strip(row))
            w,t=split(strip(wt),"|")
         #   @show w,t
            get!(w2i,w,length(w2i)+1)
             push!(words,w2i[w])
            get!(t2i,t,length(t2i)+1)
             push!(tags,t2i[t])
        end
        push!(results,(words,tags))
    end
    results
end

read_file (generic function with 1 method)

In [4]:
train = read_file("train.txt")
unk_word = get!(w2i,"<unk>",length(w2i)+1)
unk_tag =  get!(t2i,"<unk>",length(t2i)+1)
nwords = length(w2i)

words=keys(w2i)
allw_i=Array{Int64}(collect(values(w2i)))
tags=keys(t2i)
allt_i=Array{Int64}(collect(values(t2i)));
ntags = length(t2i)

dev = read_file("dev.txt");

In [5]:
struct Embed; w; end
function Embed(vocabsize::Int, embedsize::Int)
    # Your code here
    Embed(param(embedsize, vocabsize, atype = KnetArray{Float32}))
end
function (l::Embed)(x)
    # Your code here
    l.w[:,x]
end
struct Dense; w; b; f; end
Dense(i::Int,o::Int,f=identity) = Dense(param(o,i), param0(o), f)
(d::Dense)(x) = d.f.(d.w * mat(x,dims=1) .+ d.b)


In [6]:
#supervision
struct Chain
    layers
    Chain(layers...) = new(layers)
end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

In [7]:
#RL 
struct Chain2
    layers
    Chain2(layers...) = new(layers)
end
##
(c::Chain2)(x) = (for i in 1:length(c.layers)-2; x = c.layers[i](x); end; x)
(c::Chain2)(x,y) = RL_nll(c(x), y)#nll(c(x),y) 
##after calculating the nll(probs=true)-> sample over the index -> calcuate the rewards -> 

In [8]:
using Knet

In [9]:
function trainresults(file,maker,savemodel)
    if (print("Train from scratch? "); readline()[1]=='y')
        model = maker()
#         results = ((nll(model,dev), zeroone(model,dev))
        progress!(adam(model,ncycle(train,20)),seconds=60) do y               
        devloss = nll(model, dev)
        println(stderr)
        (trn_loss=nll(model,train),dev_loss=devloss,accuracy_train=1-zeroone(model,train), accuracy_dev=1-zeroone(model,dev))
        end
        Knet.save(file,"model",(savemodel ? model : nothing))
        Knet.gc()
    else
        model = Knet.load(file,"model")
    end
    return model
end

trainresults (generic function with 1 method)

In [10]:
Tagger_Supervision(vocab,embed,hidden,output)=Chain(
    Embed(vocab,embed),
    RNN(embed,hidden,rnnType=:relu,bidirectional=true),
    Dense(2hidden,output))
supervison() = Tagger_Supervision(length(w2i),EMBED_SIZE,HIDDEN_SIZE,ntags)
supervision_model = trainresults("tagger113a.jld2",supervison,true)

Train from scratch? stdin> n


Chain((Embed(P(KnetArray{Float32,2}(64,29002))), RNNRELU(input=64,hidden=128,bidirectional), Dense(P(KnetArray{Float32,2}(10,256)), P(KnetArray{Float32,1}(10)), identity)))

In [10]:
Tagger_Supervision(vocab,embed,hidden,output)=Chain(
    Embed(vocab,embed),
    RNN(embed,hidden,rnnType=:relu,bidirectional=true),
    Dense(2hidden,output))
supervison() = Tagger_Supervision(length(w2i),EMBED_SIZE,HIDDEN_SIZE,ntags)
supervision_model = trainresults("tagger113a.jld2",supervison,true)

Train from scratch? stdin> y



┣                    ┫ [0.00%, 1/200000, 00:28/1577:47:24, 28.40s/i] (trn_loss = 2.2919664f0, dev_loss = 2.2923627f0, accuracy_train = 0.8625654133640813, accuracy_dev = 0.8351321557669137)
┣█▋                  ┫ [8.53%, 17060/200000, 01:37/18:57, 248.65i/s] (trn_loss = 0.13141876f0, dev_loss = 0.45431554f0, accuracy_train = 0.9702182730065555, accuracy_dev = 0.8793037147178712)
┣███▍                ┫ [17.27%, 34550/200000, 02:46/15:59, 254.46i/s] (trn_loss = 0.15131481f0, dev_loss = 0.43436384f0, accuracy_train = 0.9570630306646563, accuracy_dev = 0.8753300689619812)
┣█████▏              ┫ [26.03%, 52065/200000, 03:54/15:00, 255.90i/s] (trn_loss = 0.14088894f0, dev_loss = 0.45307937f0, accuracy_train = 0.9626226626565284, accuracy_dev = 0.8797908067782706)
┣██████▉             ┫ [34.82%, 69647/200000, 05:03/14:29, 256.33i/s] (trn_loss = 0.17293346f0, dev_loss = 0.4608875f0, accuracy_train = 0.9532853836912086, accuracy_dev = 0.8733304278719204)
┣████████▋           ┫ [43.61%, 87226/2

Chain((Embed(P(KnetArray{Float32,2}(64,29002))), RNNRELU(input=64,hidden=128,bidirectional), Dense(P(KnetArray{Float32,2}(10,256)), P(KnetArray{Float32,1}(10)), identity)))

In [11]:
EMB,LSTM_RNN,DENSE=supervision_model.layers

(Embed(P(KnetArray{Float32,2}(64,29002))), RNNRELU(input=64,hidden=128,bidirectional), Dense(P(KnetArray{Float32,2}(10,256)), P(KnetArray{Float32,1}(10)), identity))

In [12]:
Tagger_RL(Embed_layer, Rnn, Rnn_d,hidden)=Chain2(
    Embed_layer,
    Rnn,
    Rnn_d, 
    Dense(2hidden,1))
RL() = Tagger_RL(EMB, LSTM_RNN, DENSE,HIDDEN_SIZE)
RL2 = RL()

Chain2((Embed(P(KnetArray{Float32,2}(64,29002))), RNNRELU(input=64,hidden=128,bidirectional), Dense(P(KnetArray{Float32,2}(10,256)), P(KnetArray{Float32,1}(10)), identity), Dense(P(KnetArray{Float32,2}(1,256)), P(KnetArray{Float32,1}(1)), identity)))

In [13]:
function RL_nll(scores_rnn, tags) 
    scores_rnn_per = permutedims(scores_rnn)
    # Do forward pass over the RNN_D layer.
    RNN_D = RL2.layers[length(RL2.layers)-1]
    scores = RNN_D(scores_rnn)
    
    losses,_ = nll(scores, tags; average = false)
    scores_per = permutedims(scores)
    probs = softmax(scores_per; dims = 2)
    losses= []
    for i in 1:size(scores_per)[1]
        loss,_ = nll(scores_per[i,:], [tags[i]], average = false)
        push!(losses, loss)
    end
    
    # Sample.
    samples = []
    for i in 1:size(probs)[1]
        sam = sample(collect(1:length(probs[1,:])), Weights(value(Array(probs[i,:]))))#Weights(Array(probs[i,:])))
        push!(samples, sam)
    end
    # Calculate accuracy
    r=sum(tags.==samples) /  length(tags)
    r_a = KnetArray{Float32}([Float32(r)])
    r_a = reshape(r_a,  (1,1))
    
    # Do forward pass over the Reward Dense. 
    RL_layer = RL2.layers[length(RL2.layers)]
    r_b = []
    for i in 1:size(scores_rnn_per)[1]
        r_i = RL_layer(scores_rnn_per[i,:])
        push!(r_b, r_i)
    end
    rewards_over_baseline = [r_a - x for x in r_b]
    
    baseline_scores = [(r_a-x)*(r_a-x) for x in r_b]
    
     reinforce_scores = []
     for i in 1:size(scores_per)[1]
         re_s = losses[i]*value(rewards_over_baseline[i])
         push!(reinforce_scores, re_s)
     end
       
    base_sum = sum(baseline_scores); 
    mixer_sum = sum(reinforce_scores);
    return mixer_sum[1], base_sum[1]
end

RL_nll (generic function with 1 method)

In [14]:
function model_forward(sent)
    RL2.layers[length(RL2.layers)-1](RL2(sent))
end

model_forward (generic function with 1 method)

<h3>REINFORCE Training Part </h3>

In [28]:
function optimize(model,train; lr=0.0001,seconds=60,epoch=1)
    starting_time=parse(Int,Dates.format(now(),"SS"))
    time=now()
    w1 = params(model)[[1,2,3,4]]
    w2 = params(model)[[5,6]]
    for ep in 1:epoch
        iters = length(train)
        for t = 1:iters
            L1 = @diff model(train[t][1], train[t][2])[1]
            L2 = @diff model(train[t][1], train[t][2])[2]
#             if t % 1000 == 0
#                println("Iter: ", t) 
#             end
            for i in 1:length(w1)
                ∇wi = grad(L1,w1[i])
                if ∇wi != nothing;
                     w1[i] .-= lr * AutoGrad.full(∇wi);
                end
            end
        
            for i in 1:length(w2)
                ∇wi2 = grad(L2,w2[i])
                if ∇wi2 != nothing;
                    w2[i] .-= lr * AutoGrad.full(∇wi2);
                end
            end
            if(parse(Int,Dates.format(now(),"SS"))-starting_time)%seconds==0
                println((epoch=ep,accuracy_dev=1-zeroone(model_forward,dev) , loss_trn=value(L1)+value(L2) , time=(now()-time)))
                starting_time=starting_time+60
            end
        end  
    end
    #w1,w2
end

optimize (generic function with 1 method)

In [29]:
optimize(RL2, train;seconds=60,epoch=10)

(epoch = 1, accuracy_dev = 0.8649473171482042, loss_trn = 1.2439224f0, time = 952 milliseconds)
(epoch = 1, accuracy_dev = 0.8653574999359089, loss_trn = 0.36755392f0, time = 60072 milliseconds)
(epoch = 1, accuracy_dev = 0.8653831363601405, loss_trn = 0.38101205f0, time = 60713 milliseconds)
(epoch = 1, accuracy_dev = 0.8648704078755095, loss_trn = 0.2603824f0, time = 120178 milliseconds)
(epoch = 1, accuracy_dev = 0.8648704078755095, loss_trn = 1.7023038f0, time = 120866 milliseconds)
(epoch = 1, accuracy_dev = 0.865921501269003, loss_trn = 2.2027082f0, time = 180087 milliseconds)
(epoch = 1, accuracy_dev = 0.865921501269003, loss_trn = 0.85630894f0, time = 180694 milliseconds)
(epoch = 1, accuracy_dev = 0.8664855026020971, loss_trn = 0.07008718f0, time = 240040 milliseconds)
(epoch = 1, accuracy_dev = 0.8664855026020971, loss_trn = 0.040717885f0, time = 240699 milliseconds)
(epoch = 1, accuracy_dev = 0.8677416873894429, loss_trn = 1.2411798f0, time = 300165 milliseconds)
(epoch = 1,

(epoch = 4, accuracy_dev = 0.8675622324198221, loss_trn = 0.24721563f0, time = 2460671 milliseconds)
(epoch = 4, accuracy_dev = 0.8673571410259697, loss_trn = 0.59436446f0, time = 2520128 milliseconds)
(epoch = 4, accuracy_dev = 0.8673571410259697, loss_trn = -0.1487853f0, time = 2520786 milliseconds)
(epoch = 4, accuracy_dev = 0.8672802317532751, loss_trn = 0.44957504f0, time = 2580048 milliseconds)
(epoch = 4, accuracy_dev = 0.8672802317532751, loss_trn = 0.6391899f0, time = 2580766 milliseconds)
(epoch = 4, accuracy_dev = 0.8672545953290435, loss_trn = 0.09252235f0, time = 2640021 milliseconds)
(epoch = 4, accuracy_dev = 0.867228958904812, loss_trn = -1.5117536f0, time = 2640626 milliseconds)
(epoch = 4, accuracy_dev = 0.8670238675109596, loss_trn = 0.5377617f0, time = 2700095 milliseconds)
(epoch = 4, accuracy_dev = 0.8668444125413387, loss_trn = -0.7339355f0, time = 2700716 milliseconds)
(epoch = 4, accuracy_dev = 0.8667418668444126, loss_trn = 0.176253f0, time = 2760018 milliseco

(epoch = 7, accuracy_dev = 0.8657676827236137, loss_trn = 1.0706848f0, time = 4920790 milliseconds)
(epoch = 7, accuracy_dev = 0.865690773450919, loss_trn = -0.06381108f0, time = 4980037 milliseconds)
(epoch = 7, accuracy_dev = 0.865690773450919, loss_trn = 0.2063953f0, time = 4980632 milliseconds)
(epoch = 7, accuracy_dev = 0.8649473171482042, loss_trn = 0.1562514f0, time = 5040098 milliseconds)
(epoch = 7, accuracy_dev = 0.8649473171482042, loss_trn = 0.15792933f0, time = 5040686 milliseconds)
(epoch = 7, accuracy_dev = 0.8645114979362678, loss_trn = 0.67741704f0, time = 5100121 milliseconds)
(epoch = 7, accuracy_dev = 0.8645114979362678, loss_trn = 0.15026623f0, time = 5100713 milliseconds)
(epoch = 7, accuracy_dev = 0.8641269515727946, loss_trn = 0.007943999f0, time = 5160074 milliseconds)
(epoch = 7, accuracy_dev = 0.8641269515727946, loss_trn = -0.39898464f0, time = 5160687 milliseconds)
(epoch = 8, accuracy_dev = 0.8641269515727946, loss_trn = 0.53637934f0, time = 5220100 millis

(epoch = 10, accuracy_dev = 0.8591791216961058, loss_trn = 0.07343376f0, time = 7380713 milliseconds)
